In [1]:
"""
2022
For Friktion Labs
Done by: MordantBlack (enfamil#3658)
"""

import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import schedule
import time
import threading
import glob
import os.path
from pycoingecko import CoinGeckoAPI

def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': 'BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4'}
    request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.{}'.format(request.status_code, query))

In [2]:
def extract_volt_data(csv_name):
    
    # search for existing csv files in data folder
    # REMEMBER TO CHANGE PATH DIRECTORY!
    csv_files = glob.glob(os.path.join('/Users/.../data', csv_name))
    
    # if files are non-existent, to create them with initial date range from 17-30 Dec 21
    if not csv_files:
        print(csv_name + " missing")
        query_date = '"2021-12-17", "2021-12-30"'
        if csv_name == 'volt01_master.csv':
            extract_volt01_uu(query_date, csv_files)
            action = "Volt 01 Unique Users & Tx Size"
        elif csv_name == 'volt01_master_wd.csv':
            extract_volt01_wd(query_date, csv_files)
            action = "Volt 01 Withdrawals"
        elif csv_name == 'volt02_master.csv':    
            extract_volt02_uu(query_date, csv_files)
            action = "Volt 02 Unique Users & Tx Size"
        elif csv_name == 'volt02_master_wd.csv':
            extract_volt02_wd(query_date, csv_files)
            action = "Volt 02 Withdrawals"
        
        print(datetime.utcnow().strftime('%Y-%m-%d , %H:%M:%S') + " - Running query of " + action + "...")
    
    else:
        print(csv_name + " found")
        dataframe = pd.read_csv(csv_name, index_col=0, parse_dates=True)
        
        # If csv found, calculate date range to retrieve updated data
        dataframe.index = pd.to_datetime(dataframe['date'], format = '%Y-%m-%d')
        # get 1 day before current date. This ensures that one full day of volt transactions are 
        # captured on solscan
        utc_date_bef = datetime.utcnow().date() - timedelta(days=1) # solscan is in UTC
        # df max date + 1
        start_date_query = dataframe.index.max().date() + timedelta(days=1)
        # if df max date is less than utc date, 
        if start_date_query < utc_date_bef:
            query_date =  '"' + start_date_query.strftime('%Y-%m-%d') + '"' + ',' + '"' + utc_date_bef.strftime('%Y-%m-%d') + '"'
            if csv_name == 'volt01_master.csv':
                extract_volt01_uu(query_date, csv_files)
                action = "Volt 01 Unique Users & Tx size"
            elif csv_name == 'volt01_master_wd.csv':
                extract_volt01_wd(query_date, csv_files)
                action = "Volt 01 Withdrawals"
            elif csv_name == 'volt02_master.csv':    
                extract_volt02_uu(query_date, csv_files)
                action = "Volt 02 Unique Users & Tx size"
            elif csv_name == 'volt02_master_wd.csv':    
                extract_volt02_wd(query_date, csv_files)
                action = "Volt 02 Unique Users & Tx size"
            print(datetime.utcnow().strftime('%Y-%m-%d , %H:%M:%S') + " - Running query of " + action + "...")
            
        else:
            print(datetime.utcnow().strftime('%Y-%m-%d , %H:%M:%S') +  " - No new data found, running query again in 24 hours.")
    

In [3]:
# Volt 01 Unique Users
def extract_volt01_uu(query_date, csv_files):
    # The GraphQL query - Volt01 Call
    # 16 Dec 2021 - BTC Volt 01 Call - 3BjcHXvyzMsjmeqE2qFLx45K4XFx3JPiyRnjJiF5MAHt
    # 17 Dec 2021 - SOL Volt 01 Call - 4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2
    # mSOL Volt 01 Call - 6UA3yn28XecAHLTwoCtjfzy3WcyQj1x13bxnH8urUiKt
    # ETH Volt 01 Call - GjnoPUjQiEUYWuKAbMax2cM1Eony8Yutc133wuSun9hS
    # FTT Volt 01 Call - 7wDh4VCTPwx41kvbLE6fkFgMEjnqw7NpGJvQtNabCm2B
    # SRM Volt 01 Call - 5SLqZSywodLS8ih6U2AAioZrxpgR149hR8SApmCB7r5X
    # MNGO Volt 01 Call - 4sTuzTYfcE2NF7zy6Sy8XhVcNLa6JQSLrx3roy97n4sD
    # scnSOL Volt 01 Call - 5VmdHqvRMbXivuC34w4Hux9zb1y9moiBEQmXDrTR1kV
    # SBR Volt 01 Call - DPMCwE9z9jXaDVDti5aKhdgCWGgsvioz6ZvB9eZjH7UE
    # LUNA Volt 01 Call - 95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE
    
    query = """
    query{
    solana(network: solana) {
    transfers(
      date: {between: ["""+ query_date +"""]}
      transferType: {is: transfer}
      currency: {in: ["SOL", "BTC", "mSOL", "ETH", "FTT", "SRM", "MNGO", "scnSOL", "SBR", "LUNA", "RAY"]}
      any: [{receiverAddress: {in: [
        "Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G",
        "DA1M8mw7GnPNKU9ReANtHPQyuVzKZtsuuSbCyc2uX2du",
        "6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH",
        "FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp",
        "7KqHFuUksvNhrWgoacKkqyp2RwfBNdypCYgK9nxD1d6K",
        "2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb",
        "B3yakZxwomkmnCxRr8ZmQtiWgtxtVBuCREDFDdAvcCVQ",
        "A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A",
        "BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h",
        "5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X",
        "A6XsYxGj9wpqUZG81XwgQJ2zJ3efCbuWSQfnkHqUSmdM"]}}]
    ) {
      amount
      currency {
        symbol
      }
      date {
        date
      }
      transaction {
        signer
      }
      sender {
        address
      }
    }
    }
    }
    """
    
    print("running query between dates " + query_date)
    result = run_query(query)
    # convert GraphQL json to pandas dataframe
    df = pd.json_normalize(result['data']['solana']['transfers'])
    # relabel columns
    df = df.rename(columns={"date.date": "date", "currency.symbol": "asset", "transaction.signer":"signer","sender.address":"sender"})
    # IMPORTANT - if transaction signer & receiver is diff, drop row
    df = df.query("signer == sender").reset_index(drop=True)
    
    if not csv_files:
        df.to_csv("volt01_master.csv")
        print("Master sheet created with dates between " + query_date)
    else:
        df.to_csv("volt01_master.csv", mode='a', header=False)
        print("Master sheet created with dates Updated!") 
    
    # read csv
    df_uu = pd.read_csv("volt01_master.csv", index_col=0)
    df_ts = pd.read_csv("volt01_master.csv", index_col=0)

    #---------------- UNIQUE USERS ----------------------------
    df_uu = df_uu.groupby(['asset', 'date']).size().groupby(level=0).cumsum().reset_index(name='count')
    # remove duplicated rows if any
    #df_uu.drop_duplicates(keep='first',inplace=True)
    # save Unique Users df to csv
    df_uu.to_csv("volt01_uu.csv")
    print("volt01_uu.csv updated with date range " + query_date)
    
    #---------------- TRANSACTION SIZE ----------------------------
    # sum up amount according to sender addresses by asset grp
    agg_functions = {'amount': 'sum'}
    df_ts = df_ts.groupby(['asset','sender']).agg(agg_functions).reset_index()
    # get average size of each asset grp
    agg_functions_2 = {'amount': 'mean'}
    df_ts = df_ts.groupby(['asset']).agg(agg_functions_2).reset_index()

    agg_functions = {'count': 'sum'}
    df_uu = df_uu.groupby(['asset']).agg(agg_functions).reset_index()
    df_merge = pd.merge(df_ts, df_uu, on="asset")

    # get asset prices
    cg = CoinGeckoAPI()
    # SOL, BTC, mSOL, ETH, FTT, SRM", "MNGO", "scnSOL", "SBR", "LUNA", "RAY"
    asset_price = cg.get_price(ids='solana, bitcoin, msol, ethereum, ftx-token, serum, mango-markets, socean-staked-sol, saber, terra-luna, raydium', vs_currencies='usd')
    df_price = pd.DataFrame(asset_price)
    df_price = df_price.rename(columns={"solana": "SOL", "bitcoin": "BTC", "msol": "mSOL", 
                         "ethereum": "ETH", "ftx-token": "FTT", "serum": "SRM", 
                         "mango-markets": "MNGO", "socean-staked-sol": "scnSOL", 
                         "saber": "SBR", "terra-luna": "LUNA", "raydium": "RAY"})
    df_price = df_price.T
    df_price = df_price.rename_axis('asset').reset_index()
    # Concatenate 2 tables tgt aligning assets
    final_df = pd.merge(df_merge, df_price, on='asset')
    # Get average transaction size in USD
    final_df["tx_size"] = final_df["amount"] * final_df["usd"]
    final_df.sort_values(by=['tx_size'], inplace=True)
    final_df = final_df.fillna(0)
    # save Tx Size df to csv
    final_df.to_csv("volt01_ts.csv")
    print("volt01_ts.csv updated with date range " + query_date)

In [4]:
#--------- WITHDRAWALS per Epoch from Volt 01 ------------------------------- 
def extract_volt01_wd(query_date, csv_files):
    # currency: {in: ["SOL", "BTC", "mSOL", "ETH", "FTT", "SRM", "MNGO", "scnSOL", "SBR", "LUNA", "RAY"]}

    query = """
        query{
        solana(network: solana) {
        transfers(
          date: {between: ["""+ query_date +"""]}
          transferType: {is: transfer}
          any: [{senderAddress: {in: [
            "Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G",
            "DA1M8mw7GnPNKU9ReANtHPQyuVzKZtsuuSbCyc2uX2du",
            "6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH",
            "FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp",
            "7KqHFuUksvNhrWgoacKkqyp2RwfBNdypCYgK9nxD1d6K",
            "2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb",
            "B3yakZxwomkmnCxRr8ZmQtiWgtxtVBuCREDFDdAvcCVQ",
            "A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A",
            "BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h",
            "5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X",
            "A6XsYxGj9wpqUZG81XwgQJ2zJ3efCbuWSQfnkHqUSmdM"]}}]
        ) {
          amount
          currency {
            symbol
          }
          block {
            timestamp {
              iso8601
            }
          }
          transaction {
            signer
          }
          receiver {
            address
          }
          sender {
            address
          }
        }
      }
    }
    """

    result = run_query(query)
    # convert GraphQL json to pandas dataframe
    df = pd.json_normalize(result['data']['solana']['transfers'])
    df = df.rename(columns={"block.timestamp.iso8601": "date", "currency.symbol": "symbol", 
                            "transaction.signer":"signer","receiver.address":"receiver",
                            "sender.address":"asset"})
    # if transaction signer & receiver is diff, drop row
    df = df.query("signer == receiver")
    
    if not csv_files:
        df.to_csv("volt01_master_wd.csv")
        print("Master sheet withdrawal created with dates between " + query_date)
    else:
        df.to_csv("volt01_master_wd.csv", mode='a', header=False)
        print("Master sheet withdrawal created with dates Updated!") 
    
    df_wd = pd.read_csv("volt01_master_wd.csv", index_col=0)
    
    # assign asset label over sender address
    asset_conditions = [
            (df_wd['asset'] == "Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G"),
            (df_wd['asset'] == "DA1M8mw7GnPNKU9ReANtHPQyuVzKZtsuuSbCyc2uX2du"),
            (df_wd['asset'] == "6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH"),
            (df_wd['asset'] == "FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp"),
            (df_wd['asset'] == "7KqHFuUksvNhrWgoacKkqyp2RwfBNdypCYgK9nxD1d6K"),
            (df_wd['asset'] == "2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb"),
            (df_wd['asset'] == "B3yakZxwomkmnCxRr8ZmQtiWgtxtVBuCREDFDdAvcCVQ"),
            (df_wd['asset'] == "A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A"),
            (df_wd['asset'] == "BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h"),
            (df_wd['asset'] == "5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X"),
            (df_wd['asset'] == "A6XsYxGj9wpqUZG81XwgQJ2zJ3efCbuWSQfnkHqUSmdM")
        ]
    
    # relabeling addresses as asset labels
    asset_Categories = ["SOL", "BTC", "mSOL", "ETH", "FTT", "SRM", "MNGO", "scnSOL", "SBR", "LUNA", "RAY"]
    df_wd['asset'] = np.select(asset_conditions, asset_Categories)
    # drop redundant columns
    df_wd.drop(['symbol', 'signer', 'receiver'], axis=1, inplace=True)
    df_wd['date'] = pd.to_datetime(df_wd['date'], format = '%Y-%m-%dT%H:%M:%SZ')
    # groupby date in grps of 1 week & sum amount of each asset
    df_wd = df_wd.groupby([pd.Grouper(key='date', freq='7D'),'asset']).sum()
    # Adding Epoch labels per week
    df_wd = df_wd.reset_index()
    date_df = pd.DataFrame(df_wd['date'].unique())
    epoch_lst = []
    for i in range(len(date_df)):
        epoch_lst.append('Epoch ' + str(i + 1) + ' - ' + str(date_df[0][i].strftime('%Y-%m-%d')))
    
    epoch_df = pd.DataFrame(list(zip(date_df[0], epoch_lst)), columns =['date', 'epoch'])
    merge_on_date = pd.merge(epoch_df, df_wd[["date","asset","amount"]], on="date")
    # get price of assets
    cg = CoinGeckoAPI()

    # SOL, BTC, mSOL, ETH, FTT, SRM", "MNGO", "scnSOL", "SBR", "LUNA", "RAY"
    asset_price = cg.get_price(ids='solana, bitcoin, msol, ethereum, ftx-token, serum, mango-markets, socean-staked-sol, saber, terra-luna, raydium', vs_currencies='usd')
    df_price = pd.DataFrame(asset_price)
    df_price = df_price.rename(columns={"solana": "SOL", "bitcoin": "BTC", "msol": "mSOL", 
                             "ethereum": "ETH", "ftx-token": "FTT", "serum": "SRM", 
                             "mango-markets": "MNGO", "socean-staked-sol": "scnSOL", 
                             "saber": "SBR", "terra-luna": "LUNA", "raydium": "RAY"})

    df_price = df_price.T
    # rename axis & merge with other df at asset column
    df_price = df_price.rename_axis('asset').reset_index()
    # concat both dataframes tgt
    merge = pd.merge(df_price, merge_on_date[["epoch","asset","amount"]], on="asset")
    merge['amt_usd'] = merge['amount'] * merge['usd']
    
    # save df to csv
    merge.to_csv("volt01_wd.csv")
    print("volt01_wd.csv Updated with date range " + query_date)

In [5]:
def extract_volt02_uu(query_date, csv_files):
    
    # Volt 02 SOL Put: 6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK
    # Volt 02 BTC Put: GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f
    # Volt 02 SOL Put (tsUSDC): AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8
    # Volt 02 ETH Put: EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax
    # Volt 02 LUNA Put: 5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj
    # Volt 02 MNGO Put: CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6
    query = """
    query{
        solana(network: solana) {
        transfers(
          transferType: {is: transfer}
          date: {between: ["""+ query_date +"""]}
          any: [{receiverAddress: {in: [
            "6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK",
            "GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f",
            "AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8",
            "EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax",
            "5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj",
            "CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6"]}}]
        ) {
          amount
          currency {
            symbol
            address
          }
          date {
            date
          }
          transaction {
            signer
          }
          sender {
            address
          }
          receiver {
            address
          }
        }
      }
    }
    """

    result = run_query(query)
    # convert GraphQL json to pandas dataframe
    df = pd.json_normalize(result['data']['solana']['transfers'])

    df = df.rename(columns={"date.date": "date", "currency.symbol": "symbol", 
                            "transaction.signer":"signer", "currency.address":"cash",
                            "sender.address":"sender", "receiver.address":"asset"})
    # if transaction signer & receiver is diff, drop row
    df = df.query("signer == sender").reset_index(drop=True)

    # assign asset label over address
    # UST: 9vMJfxuKxXBoEa7rM12mYLMwTacLMLDJqHozw96WQL8i
    # tsUSDC: Cvvh8nsKZet59nsDDo3orMa3rZnPWQhpgrMCVcRDRgip
    # USDC: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
    cash_conditions = [
            (df['cash'] == "9vMJfxuKxXBoEa7rM12mYLMwTacLMLDJqHozw96WQL8i"),
            (df['cash'] == "Cvvh8nsKZet59nsDDo3orMa3rZnPWQhpgrMCVcRDRgip"),
            (df['cash'] == "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"),
        ]

    # relabeling addresses as asset labels
    cash_Categories = ["UST", "tsUSDC", "USDC"]
    df['cash'] = np.select(cash_conditions, cash_Categories)
    # remove non-stablecoin assets
    df = df[df["cash"] != "0"]

    # Volt 02 SOL Put: 6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK
    # Volt 02 BTC Put: GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f
    # Volt 02 SOL Put (tsUSDC): AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8
    # Volt 02 ETH Put: EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax
    # Volt 02 LUNA Put: 5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj
    # Volt 02 MNGO Put: CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6

    asset_conditions = [
            (df['asset'] == "6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK"),
            (df['asset'] == "GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f"),
            (df['asset'] == "AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8"),
            (df['asset'] == "EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax"),
            (df['asset'] == "5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj"),
            (df['asset'] == "CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6"),
        ]

    # relabeling addresses as asset labels
    asset_Categories = ["SOL", "BTC", "SOL (tsUSDC)", "ETH", "LUNA", "MNGO"]
    df['asset'] = np.select(asset_conditions, asset_Categories)

    if not csv_files:
        df.to_csv("volt02_master.csv")
        print("Volt 02 Master sheet created with dates between " + query_date)
    else:
        df.to_csv("volt02_master.csv", mode='a', header=False)
        print("Volt 02 Master sheet created with dates Updated!") 

    # # read csv
    df_uu = pd.read_csv("volt02_master.csv", index_col=0)
    df_ts = pd.read_csv("volt02_master.csv", index_col=0)

    #---------------- UNIQUE USERS ----------------------------
    df_uu = df_uu.groupby(['asset', 'date']).size().groupby(level=0).cumsum().reset_index(name='count')
    # remove duplicated rows if any
    #df_uu.drop_duplicates(keep='first',inplace=True)
    # save unique users df to csv
    df_uu.to_csv("volt02_uu.csv")
    print("volt02_uu.csv updated with date range " + query_date)

    #---------------- TRANSACTION SIZE ----------------------------
    # sum up amount according to sender addresses by asset grp
    agg_functions = {'amount': 'sum'}
    df_ts = df_ts.groupby(['asset','sender']).agg(agg_functions).reset_index()
    # get average size of each asset grp
    agg_functions_2 = {'amount': 'mean'}
    df_ts = df_ts.groupby(['asset']).agg(agg_functions_2).reset_index()

    agg_functions = {'count': 'sum'}
    df_uu = df_uu.groupby(['asset']).agg(agg_functions).reset_index()
    df_merge = pd.merge(df_ts, df_uu, on="asset")

    df_merge['tx_size'] = df_merge["amount"] * 1
    # save Tx Size df to csv
    df_merge.to_csv("volt02_ts.csv")
    print("volt02_ts.csv updated with date range " + query_date)

In [6]:
#--------- WITHDRAWALS per Epoch from Volt 02 ------------------------------- 
def extract_volt02_wd(query_date, csv_files):
    # Volt 02 SOL Put: 6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK
    # Volt 02 BTC Put: GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f
    # Volt 02 SOL Put (tsUSDC): AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8
    # Volt 02 ETH Put: EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax
    # Volt 02 LUNA Put: 5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj
    # Volt 02 MNGO Put: CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6

    query = """
        query{
        solana(network: solana) {
        transfers(
          date: {between: ["""+ query_date +"""]}
          transferType: {is: transfer}
          any: [{senderAddress: {in: [
            "6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK",
            "GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f",
            "AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8",
            "EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax",
            "5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj",
            "CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6"]}}]
        ) {
          amount
          currency {
            symbol
            address
          }
          block {
            timestamp {
              iso8601
            }
          }
          transaction {
            signer
          }
          receiver {
            address
          }
          sender {
            address
          }
        }
      }
    }
    """

    result = run_query(query)
    # convert GraphQL json to pandas dataframe
    df = pd.json_normalize(result['data']['solana']['transfers'])
    df = df.rename(columns={"block.timestamp.iso8601": "date", "currency.symbol": "symbol", 
                            "transaction.signer":"signer", "currency.address":"cash",
                            "receiver.address":"receiver", "sender.address":"asset"})
    # if transaction signer & receiver is diff, drop row
    df = df.query("signer == receiver").reset_index(drop=True)
    # assign asset label over address
    # UST: 9vMJfxuKxXBoEa7rM12mYLMwTacLMLDJqHozw96WQL8i
    # tsUSDC: Cvvh8nsKZet59nsDDo3orMa3rZnPWQhpgrMCVcRDRgip
    # USDC: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v
    cash_conditions = [
            (df['cash'] == "9vMJfxuKxXBoEa7rM12mYLMwTacLMLDJqHozw96WQL8i"),
            (df['cash'] == "Cvvh8nsKZet59nsDDo3orMa3rZnPWQhpgrMCVcRDRgip"),
            (df['cash'] == "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"),
        ]

    # relabeling addresses as asset labels
    cash_Categories = ["UST", "tsUSDC", "USDC"]
    df['cash'] = np.select(cash_conditions, cash_Categories)
    # remove non-stablecoin assets
    df = df[df["cash"] != "0"]
    
    # Volt 02 SOL Put: 6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK
    # Volt 02 BTC Put: GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f
    # Volt 02 SOL Put (tsUSDC): AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8
    # Volt 02 ETH Put: EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax
    # Volt 02 LUNA Put: 5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj
    # Volt 02 MNGO Put: CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6
    
    asset_conditions = [
            (df['asset'] == "6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK"),
            (df['asset'] == "GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f"),
            (df['asset'] == "AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8"),
            (df['asset'] == "EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax"),
            (df['asset'] == "5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj"),
            (df['asset'] == "CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6"),
        ]

    # relabeling addresses as asset labels
    asset_Categories = ["SOL", "BTC", "SOL (tsUSDC)", "ETH", "LUNA", "MNGO"]
    df['asset'] = np.select(asset_conditions, asset_Categories)
    
    
    # save into csv
    if not csv_files:
        df.to_csv("volt02_master_wd.csv")
        print("Volt 02 Master sheet withdrawal created with dates between " + query_date)
    else:
        df.to_csv("volt02_master_wd.csv", mode='a', header=False)
        print("Volt 02 Master sheet withdrawal created with dates Updated!") 
    
    # read csv
    df_wd = pd.read_csv("volt02_master_wd.csv", index_col=0)
    
    # drop redundant columns
    df_wd.drop(['symbol', 'signer', 'receiver'], axis=1, inplace=True)
    df_wd['date'] = pd.to_datetime(df_wd['date'], format = '%Y-%m-%dT%H:%M:%SZ')
    # groupby date in grps of 1 week & sum amount of each asset
    df_wd = df_wd.groupby([pd.Grouper(key='date', freq='7D'),'asset']).sum()
    # Adding Epoch labels per week
    df_wd = df_wd.reset_index()
    date_df = pd.DataFrame(df_wd['date'].unique())
    epoch_lst = []
    for i in range(len(date_df)):
        epoch_lst.append('Epoch ' + str(i + 1) + ' - ' + str(date_df[0][i].strftime('%Y-%m-%d')))

    epoch_df = pd.DataFrame(list(zip(date_df[0], epoch_lst)), columns =['date', 'epoch'])
    merge_on_date = pd.merge(epoch_df, df_wd[["date","amount","asset"]], on="date")
    merge_on_date['amt_usd'] = merge_on_date['amount'] * 1
    merge_on_date
    # save df to csv
    merge_on_date.to_csv("volt02_wd.csv")
    print("volt02_wd.csv Updated with date range " + query_date)

In [7]:
# If want files to be immediately generated for date range 17-30 Dec 22, use below lines. 
# Repeat again to generate data to latest date

In [8]:
# Will need to manually keep in date range under query_date if run into errors
extract_volt_data("volt01_master.csv")

volt01_master.csv found
running query between dates "2022-02-19","2022-03-16"
Master sheet created with dates Updated!
volt01_uu.csv updated with date range "2022-02-19","2022-03-16"
volt01_ts.csv updated with date range "2022-02-19","2022-03-16"
2022-03-17 , 04:39:52 - Running query of Volt 01 Unique Users & Tx size...


In [9]:
extract_volt_data("volt01_master_wd.csv")

volt01_master_wd.csv found
Master sheet withdrawal created with dates Updated!
volt01_wd.csv Updated with date range "2022-02-19","2022-03-16"
2022-03-17 , 04:40:10 - Running query of Volt 01 Withdrawals...


In [10]:
extract_volt_data("volt02_master.csv")

volt02_master.csv found
Volt 02 Master sheet created with dates Updated!
volt02_uu.csv updated with date range "2022-02-19","2022-03-16"
volt02_ts.csv updated with date range "2022-02-19","2022-03-16"
2022-03-17 , 04:40:28 - Running query of Volt 02 Unique Users & Tx size...


In [11]:
extract_volt_data("volt02_master_wd.csv")

volt02_master_wd.csv found
Volt 02 Master sheet withdrawal created with dates Updated!
volt02_wd.csv Updated with date range "2022-02-19","2022-03-16"
2022-03-17 , 04:40:41 - Running query of Volt 02 Unique Users & Tx size...


In [12]:
# If prefer to set a schedule for updating, below will run everyday at a certain time of your specification. 
# Leaving 2 minutes inbetween each function 
# when using .do, argument within function is placed in the following manner
schedule.every().day.at("01:09").do(extract_volt_data, "volt01_master.csv")
schedule.every().day.at("01:11").do(extract_volt_data, "volt01_master_wd.csv")
schedule.every().day.at("01:13").do(extract_volt_data, "volt02_master.csv")
schedule.every().day.at("01:15").do(extract_volt_data, "volt02_master_wd.csv")

while True:
    # Checks whether a scheduled task 
    # is pending to run or not
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 